In [55]:
import numpy as np
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes, title=None, cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=[14, 10])
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=12)
    plt.yticks(tick_marks, classes, fontsize=12)
    
    if title:
        plt.title(title)

    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], '.2f') + '%',
                     ha="center", va="center",
                     fontsize=14,
                     color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    
    plt.xlabel('Predicted label', fontsize=14)
    plt.ylabel('True label', fontsize=14)



In [56]:
# Copyright 2018 Xanadu Quantum Technologies Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


"""Script for creating Plots"""
import numpy as np
import matplotlib.pyplot as plt
#import plot_confusion_matrix
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

plt.switch_backend('agg')

# Label for simulation
simulation_label = 1

# Loading confusion table
confusion_table = np.load('/content/confusion_table.npy')

# Defining array of thresholds from 0 to 1 to consider in the ROC curve
thresholds_points = 101
thresholds = np.linspace(0, 1, num=thresholds_points)

# false/true positive/negative rates
fp_rate = []
tp_rate = []
fn_rate = []
tn_rate = []

# Creating rates
for i in range(thresholds_points):
    fp_rate.append(confusion_table[i, 0, 1] / (confusion_table[i, 0, 1] + confusion_table[i, 0, 0]))
    tp_rate.append(confusion_table[i, 1, 1] / (confusion_table[i, 1, 1] + confusion_table[i, 1, 0]))

    fn_rate.append(confusion_table[i, 1, 0] / (confusion_table[i, 1, 1] + confusion_table[i, 1, 0]))
    tn_rate.append(confusion_table[i, 0, 0] / (confusion_table[i, 0, 0] + confusion_table[i, 0, 1]))

# Distance of each threshold from ideal point at (0, 1)
distance_from_ideal = (np.array(tn_rate) - 1)**2 + (np.array(fn_rate) - 0)**2

# Threshold closest to (0, 1)
closest_threshold = np.argmin(distance_from_ideal)

# Area under ROC curve
area_under_curve = np.trapz(np.sort(tn_rate), x=np.sort(fn_rate))

print("Area under ROC curve: " + str(area_under_curve))
print("Closest threshold to optimal ROC: " + str(thresholds[closest_threshold]))

# Plotting ROC curve
straight_line = np.linspace(0, 1, 1001)

plt.figure(figsize=(10,7))
plt.gcf().subplots_adjust(bottom=0.15)

plt.gcf().subplots_adjust(bottom=0.15)
plt.plot(fn_rate, tn_rate, label='NN+Encoder+QNN', color='#056eee', linewidth=2.2)
plt.plot(straight_line, straight_line, color='#070d0d', linewidth=1.5, dashes=[6, 2])
plt.annotate('Minimum ROC Score of 50%\n(This is the minimum score to get)', xy=(0.7, 0.7), xytext=(0.6, 0.5),
            arrowprops=dict(facecolor='#6E726D', shrink=0.05),
            )  
plt.plot(0.0, 1.0, 'ko')
plt.plot(fn_rate[closest_threshold], tn_rate[closest_threshold], 'k^')
plt.ylim(-0.05, 1.05)
plt.xlim(-0.05, 1.05)
plt.grid(True)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.title("ROC Curve For NN+Encoder+QNN", fontsize=16)
plt.xlabel('False Negative Rate', fontsize=15)
plt.ylabel('True Negative Rate', fontsize=15)
plt.tick_params(axis='both', which='major', labelsize=12, length=6, width=1)
plt.tick_params(axis='both', which='minor', labelsize=12, length=6, width=1)
plt.legend(loc='lower right');

plt.savefig('./roc.png')
plt.show()
#plt.close()

# Selecting ideal confusion table and plotting
confusion_table_ideal = confusion_table[closest_threshold]


plot_confusion_matrix(confusion_table_ideal, classes=['No Backorder', 'Backorder'], title='')

plt.savefig('./confusion.png')
plt.show()

Area under ROC curve: 0.7108582089552239
Closest threshold to optimal ROC: 0.54


In [59]:
confusion_table_ideal

array([[59.17602996, 15.73033708],
       [ 7.11610487, 17.97752809]])

In [62]:
import numpy as np
from sklearn.metrics import classification_report
from imblearn.metrics import classification_report_imbalanced

# Define the confusion matrix in percentage format
confusion_matrix_percent = np.array([[59.17602996, 15.73033708],
                                     [7.11610487, 17.97752809]])

# Define the instance counts
instance_counts = np.array([67, 200])

# Convert the confusion matrix percentages to counts
confusion_matrix_count = np.round(confusion_matrix_percent / 100 * np.sum(instance_counts))
confusion_matrix_count

array([[158.,  42.],
       [ 19.,  48.]])

In [64]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, balanced_accuracy_score, classification_report

# Define the confusion matrix
confusion_matrix_freq = np.array([[158., 42.],
                                 [19., 48.]])

# Calculate the metrics
true_positive = confusion_matrix_freq[1, 1]
false_positive = confusion_matrix_freq[0, 1]
false_negative = confusion_matrix_freq[1, 0]
true_negative = confusion_matrix_freq[0, 0]

precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
f1 = 2 * (precision * recall) / (precision + recall)
sensitivity = recall
specificity = true_negative / (true_negative + false_positive)
accuracy = (true_positive + true_negative) / (true_positive + false_positive + false_negative + true_negative)

macro_avg_precision = precision_score([0, 1], [0, 1], average='macro', zero_division=0)
macro_avg_recall = recall_score([0, 1], [0, 1], average='macro', zero_division=0)
macro_avg_f1 = f1_score([0, 1], [0, 1], average='macro', zero_division=0)

weighted_avg_precision = precision_score([0, 1], [0, 1], average='weighted', zero_division=0)
weighted_avg_recall = recall_score([0, 1], [0, 1], average='weighted', zero_division=0)
weighted_avg_f1 = f1_score([0, 1], [0, 1], average='weighted', zero_division=0)

geometric_mean = np.sqrt(sensitivity * specificity)

iba = balanced_accuracy_score([0, 1], [0, 1])

# Print the metrics with 2 decimal places
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 Score: {:.2f}".format(f1))
print("Sensitivity: {:.2f}".format(sensitivity))
print("Specificity: {:.2f}".format(specificity))
print("Accuracy: {:.2f}".format(accuracy))
print("Macro Average Precision: {:.2f}".format(macro_avg_precision))
print("Macro Average Recall: {:.2f}".format(macro_avg_recall))
print("Macro Average F1 Score: {:.2f}".format(macro_avg_f1))
print("Weighted Average Precision: {:.2f}".format(weighted_avg_precision))
print("Weighted Average Recall: {:.2f}".format(weighted_avg_recall))
print("Weighted Average F1 Score: {:.2f}".format(weighted_avg_f1))
print("Geometric Mean: {:.2f}".format(geometric_mean))
print("IBA (Index of Balanced Accuracy): {:.2f}".format(iba))


Precision: 0.53
Recall: 0.72
F1 Score: 0.61
Sensitivity: 0.72
Specificity: 0.79
Accuracy: 0.77
Macro Average Precision: 1.00
Macro Average Recall: 1.00
Macro Average F1 Score: 1.00
Weighted Average Precision: 1.00
Weighted Average Recall: 1.00
Weighted Average F1 Score: 1.00
Geometric Mean: 0.75
IBA (Index of Balanced Accuracy): 1.00
